# Chapter 7: Graph Traversal

## Simulating Graph Algorithms

In [1]:
# Example
# V = 3 (Number of vertices)
# edges = [(0, 1), (1, 2), (2, 0)]
def create_adjacency_matrix(V, edges, undirected = True):
    # Initialize an empty V x V matrix with all zeros
    matrix = [[0] * V for _ in range(V)]
    
    # Populate the matrix based on the edges
    for edge in edges:
        u, v = edge
        matrix[u][v] = 1
        if undirected:
            matrix[v][u] = 1
    
    return matrix

def create_adjacency_list(V, edges, undirected = True):
   
    # Add vertices to the dictionary
    adjacency_list = {i: [] for i in range(V)}

    # Add edges to the dictionary
    for edge in edges:
        u, v = edge
        adjacency_list[u].append(v)
        if undirected:
            adjacency_list[v].append(u)

    return adjacency_list

### 7-4
Prove that in a breadth-first search on a undirected graph $G$, every edge is
either a tree edge or a cross edge, where $x$ is neither an ancestor nor descendant
of $y$ in cross edge $(x, y)$.


> When doing BFS from a given starting vertex, every neighbour is added to a queue after the discovery. Later the newly discovered vertices are processed in FIFO manner, the oldest discovered vertex is processed first. As a consequence the resulting BFS tree given after traversing $G$ has the property that the tree edges cover the shortest possible path between any two vertices.

> Now suppose $x$ and $y$ is neither a cross edge or tree edge, and $x$ is a descendant of $y$. That means $y$ is discovered first, and since $x$ is a child, it should be discovered at the next level. But that would mean the edge $(x, y)$ is a tree edge. So by contradiction we can conclude that the original statement cannot be true.

### 7-8
Given pre-order and in-order traversals of a binary tree, is it possible to reconstruct the tree? If so, sketch an algorithm to do it.
If not, give a counterexample. Repeat the problem if you are given the pre-order
and post-order traversals.
> First of all, let's start with the second question, pre-order and post-order traversals. I can show with a counterexample that it is not possible to reconstruct the tree. Let's consider a fairly simple tree with two nodes, where node A is the root and node B is the left child:
> * pre-order traversal is A, B
> * post-order traversal is B, A

> Now what happens if I have a similar tree with the root node being A but this time B is a right child?
> * pre-order traversal is A, B
> * post-order traversal is B, A

> Looks like it is completely the same for the both pre-order and post-order traversals, meaning that reconstructing the tree is not possible this way.

However, it is possible when given the pre-order and in-order traversals, as can be see below:

In [2]:
class Node:
    def __init__(self, key):
        self.left = None
        self.right = None
        self.val = key

print(r'''Let us construct below tree:
          1
         / \
        2   3
       / \   \
      4   5   6
''')

root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)
root.right.right = Node(6)

def traverse_pre_order(tree):
    if tree is None:
        return []
    
    return [tree.val] \
        + traverse_pre_order(tree.left) \
        + traverse_pre_order(tree.right)


def traverse_in_order(tree):
    if tree is None:
        return []

    return traverse_in_order(tree.left) \
        + [tree.val] \
        + traverse_in_order(tree.right)


def reconstruct(pre_order, in_order):
    if not pre_order:
        return None
        
    root = Node(pre_order.pop(0))
    root_index = in_order.index(root.val)
    
    in_order_left = in_order[:root_index]
    pre_order_left = [val for val in pre_order if val in in_order_left]
    left_child = reconstruct(pre_order_left, in_order_left)

    in_order_right = in_order[root_index + 1:]
    pre_order_right = [val for val in pre_order if val in in_order_right]
    right_child = reconstruct(pre_order_right, in_order_right)

    root.left = left_child
    root.right = right_child
    return root
        

pre_order = traverse_pre_order(root)
print(f"pre-order traversal: {pre_order}")

in_order = traverse_in_order(root)
print(f"in-order traversal: {in_order}")

new_root = reconstruct(pre_order, in_order)

pre_order = traverse_pre_order(new_root)
print(f"pre-order traversal from reconstructed tree: {pre_order}")

in_order = traverse_in_order(new_root)
print(f"in-order traversal from reconstructed tree: {in_order}")


Let us construct below tree:
          1
         / \
        2   3
       / \   \
      4   5   6

pre-order traversal: [1, 2, 4, 5, 3, 6]
in-order traversal: [4, 2, 5, 1, 3, 6]
pre-order traversal from reconstructed tree: [1, 2, 4, 5, 3, 6]
in-order traversal from reconstructed tree: [4, 2, 5, 1, 3, 6]


### 7-9
Present correct and efficient algorithms to convert an undirected graph $G$
between the following graph data structures. Give the time complexity of each
algorithm, assuming $n$ vertices and $m$ edges.
1. Convert from an adjacency matrix to adjacency lists.


In [3]:
def from_adjacency_matrix_to_adjacency_list(adjacency_matrix):

    # Add vertices to the dictionary
    adjacency_list = {i: [] for i in range(len(adjacency_matrix))}

    for i, row in enumerate(adjacency_matrix):
        for j, is_edge in enumerate(row):
            if is_edge == 1:
                adjacency_list[i].append(j)

    return adjacency_list



2. Convert from an adjacency list representation to an incidence matrix. An
incidence matrix $M$ has a row for each vertex and a column for each edge,
such that $M [i, j] = 1$ if vertex $i$ is part of edge $j$, otherwise $M [i, j] = 0$

In [4]:
def from_adjacency_list_to_incidence_matrix(adjacency_list):

    # Initialize an empty V x E matrix with all zeros
    V = len(adjacency_list)
    # divide by 2 because edge (a, b) is the same as (b, a) in an undirected graph
    E = len([j for i in adjacency_list.values() for j in i]) // 2
    matrix = [[0] * E for _ in range(V)]

    vertexCount = 0
    for vertex, edges in adjacency_list.items():
        for edge in edges:
            if edge > vertex:
                matrix[vertex][vertexCount] = 1
                matrix[edge][vertexCount] = 1
                vertexCount += 1

    return matrix

3. Convert from an incidence matrix to adjacency lists.

In [5]:
def from_incidence_matrix_to_adjacency_list(incidence_matrix):

    V = len(incidence_matrix)
    E = len(incidence_matrix[0])

    # Add vertices to the dictionary
    adjacency_list = {i: [] for i in range(len(incidence_matrix))}   

    for edge_idx in range(E):
        # retrieve vertex pairs (indices) composing the current edge
        # e.g. [0, 1, 0, 0, 1] -> [1, 4]
        u, v = [i for i in range(V) if incidence_matrix[i][edge_idx] == 1]
        # add edges to the adjacency list
        adjacency_list[u].append(v)
        adjacency_list[v].append(u)

    return adjacency_list
    
    

In [6]:
# testing
V = 4
edges = [(0, 1), (1, 2), (1, 3), (2, 3), (3, 0)]

am = create_adjacency_matrix(V, edges)
print(f"adjacency_matrix: {am}")

al = from_adjacency_matrix_to_adjacency_list(am)
print(f"adjacency_list: {al}")

im = from_adjacency_list_to_incidence_matrix(al)
print(f"incidence_matrix: {im}")

al = from_incidence_matrix_to_adjacency_list(im)
print(f"adjacency_list: {al}")

adjacency_matrix: [[0, 1, 0, 1], [1, 0, 1, 1], [0, 1, 0, 1], [1, 1, 1, 0]]
adjacency_list: {0: [1, 3], 1: [0, 2, 3], 2: [1, 3], 3: [0, 1, 2]}
incidence_matrix: [[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1]]
adjacency_list: {0: [1, 3], 1: [0, 2, 3], 2: [1, 3], 3: [0, 1, 2]}


### 7-16
The *square* of a directed graph $G = (V, E)$ is the graph $G^2 = (V, E^2)$ such
that $(u, w) \in E^2$ if there exists $v \in V$ such that $(u, v) \in E$ and $(v, w) \in E$; that
is, there is a path of exactly two edges from $u$ to $w$.
Give efficient algorithms for both adjacency lists and matrices.

In [7]:
def create_square_graph_from_list(adjacency_list):
    square_graph = {i: [] for i in range(len(adjacency_list))}
    
    for u in range(len(adjacency_list)):
        for v in adjacency_list[u]:
            for w in adjacency_list[v]:
                square_graph[u].append(w)

    return square_graph

def create_square_graph_from_matrix(adjacency_matrix):
    # Initialize an empty V x V matrix with all zeros
    V = len(adjacency_matrix)
    square_graph = [[0] * V for _ in range(V)]

    for u in range(V):
        for v, is_edge_v in enumerate(adjacency_matrix[u]):
            for w, is_edge_w in enumerate(adjacency_matrix[v]):
                if is_edge_v == 1 and is_edge_w == 1:
                   square_graph[u][w] = 1 

    return square_graph

In [8]:
# testing
V = 5 # (Number of vertices)
edges = [(0,3), (0, 2), (1, 0), (2, 1), (3, 4), (4, 0)]
print(f"test graph, V:  {V}, edges: {edges}")

sg = create_square_graph_from_list(create_adjacency_list(V, edges, undirected = False))
sg2 = create_square_graph_from_matrix(create_adjacency_matrix(V, edges, undirected = False))

print()
print(f"created square graph in adjacency list form:\n{sg}")
print()
print(f"created square graph in adjacency matrix form:\n{sg2}")

test graph, V:  5, edges: [(0, 3), (0, 2), (1, 0), (2, 1), (3, 4), (4, 0)]

created square graph in adjacency list form:
{0: [4, 1], 1: [3, 2], 2: [0], 3: [0], 4: [3, 2]}

created square graph in adjacency matrix form:
[[0, 1, 0, 0, 1], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0]]


### 7-17
A *vertex cover* of a graph $G = (V, E)$ is a subset of vertices $V'$ such that
each edge in $E$ is incident to at least one vertex of $V'$.
1. Give an efficient algorithm to find a minimum-size vertex cover if $G$ is a
tree.

In [9]:
class Node:
    def __init__(self, key):
        self.val = key
        self.children = []
        self.parent = None

def find_vertex_cover(root):
    """
    In a vertex cover we need to have at least one vertex for each edge.
    Every tree has at least two leaves, meaning that there is always an vertex v
    which is adjacent to a leaf u. Apparently it’s always better to choose v than u,
    since it is the only one which can also cover other edges! After trimming othe
    covered edges, we have a smaller tree. We can repeat the process until the
    tree has 0 or 1 edges. When the tree has only one isolated edge, pick either
    vertex. All leaves can be identied and trimmed in O(n) time during a DFS, so
    it takes O(n) in total.
    """
    vertex_cover = set()

    def trim_leaves(node):
        if not node.children: # is a leaf
            if not node.val in vertex_cover:
                vertex_cover.add(node.parent.val)
            # remove node from parent's children list
            node.parent.children = [c for c in node.parent.children if c is not node]
            return

        for child in node.children:
            trim_leaves(child)
    
    if not root:
        return vertex_cover

    # 1 vertex tree
    if not root.children:
        vertex_cover.add(root.val)
        return vertex_cover

    # trim the tree until only the root remains
    while root.children:
        trim_leaves(root)
    
    return vertex_cover


In [10]:
# Testing
print(r'''Test with the below tree:
            10
           /  \
         20    30
        /  \     \
      40    50    60
            /  \
          70    80
''')
tree = Node(10)
tree.children = [Node(20), Node(30)]
for child in tree.children:
    child.parent = tree 

tree.children[0].children = [Node(40), Node(50)]
for c in tree.children[0].children:
    c.parent = tree.children[0]

tree.children[0].children[1].children = [Node(70), Node(80)]
for c in tree.children[0].children[1].children:
    c.parent = tree.children[0].children[1]


tree.children[1].children = [Node(60)]
for c in tree.children[1].children:
    c.parent = tree.children[1]


print(f"Min vertex cover is : {find_vertex_cover(tree)}") 

Test with the below tree:
            10
           /  \
         20    30
        /  \     \
      40    50    60
            /  \
          70    80

Min vertex cover is : {50, 20, 30}


2. Let $G = (V, E)$ be a tree such that the weight of each vertex is equal to the
degree of that vertex. Give an efficient algorithm to find a minimum-weight
vertex cover of $G$.

In [11]:
def find_weighted_vertex_cover(root):

    vertex_cover = set()

    if not root:
        return vertex_cover
        
    """
    Let’s define score[u][include] as the minimum weight vertex cover for the
    subtree rooted at u. include ∈ {0, 1} and indicates whether we include u
    itself in the vertex cover or not.
    """
    include_map = {}

    def dfs(node):
        if not node.children: # is a leaf
            not_include_score = 0 # if not included 0 weight
            include_score = 1 # if included 1 weight since it is a leaf
            include_map[node.val] = (not_include_score, include_score)
            return

        # not a leaf node, first traverse all the children
        for child in node.children:
            dfs(child)

        # if node is not included we must include its children
        not_include_score = sum(include_map[child.val][1] for child in node.children)
        # if node is included we take the min of each child since we can choose to include or not
        # and add it to the weight of the current node
        node_weight = len(node.children) + (0 if node.parent is None else 1)
        include_score = node_weight +\
            sum(min(include_map[child.val][0], include_map[child.val][1]) for child in node.children)
        
        include_map[node.val] = (not_include_score, include_score)

    # Now the include_matrix is filled, we just have to traverse once more to get the vertex cover
    dfs(root)

    def collect_vertex_cover(node, take_min):
        """
        Collect the vertex cover from the include map. Either take the
        current node if take_min is false, otherwise the action is based on
        the min value of the current node's include or not include score.
        """

        if not node:
            return

        not_include_score, include_score = include_map[node.val]
        should_include = (take_min is False) \
            or (True if include_score < not_include_score else False)
        
        if should_include:
            vertex_cover.add(node.val)
            for child in node.children:
                collect_vertex_cover(child, take_min = True)
        else:
            for child in node.children:
                collect_vertex_cover(child, take_min = False)

    
    collect_vertex_cover(root, take_min = True)
    return vertex_cover

In [12]:
# Testing
print(r'''Test with the below tree:
            10
           /  \
         20    30
        /  \     \
      40    50    60
            /  \
          70    80
''')
tree = Node(10)
tree.children = [Node(20), Node(30)]
for child in tree.children:
    child.parent = tree 

tree.children[0].children = [Node(40), Node(50)]
for c in tree.children[0].children:
    c.parent = tree.children[0]

tree.children[0].children[1].children = [Node(70), Node(80)]
for c in tree.children[0].children[1].children:
    c.parent = tree.children[0].children[1]


tree.children[1].children = [Node(60)]
for c in tree.children[1].children:
    c.parent = tree.children[1]


print(f"Min vertex cover is : {find_weighted_vertex_cover(tree)}") 


Test with the below tree:
            10
           /  \
         20    30
        /  \     \
      40    50    60
            /  \
          70    80

Min vertex cover is : {80, 20, 70, 30}


3. Let $G = (V, E)$ be a tree with arbitrary weights associated with the vertices. Give an efficient algorithm to find a minimum-weight vertex cover
of $G$.

> The algorithm is exactly the same as for the previous exercise. Instead of assigning weights based on the vertex degree, we can use arbitrary values by extending the Node data structure. Changing the code to calculate the include score based on the the new weight field is trivial, not included here.

### 7-18
A *vertex cover* of a graph $G = (V, E)$ is a subset of vertices $V'$ such that
each edge in $E$ is incident to at least one vertex of $V'$. Delete all the leaves from any
depth-first search tree of $G$. Must the remaining vertices form a vertex cover of
$G$? Give a proof or a counterexample.

> In short, if the tree has more than one vertex, then yes. The remaining vertices are still the vertex cover because for every edge $e\in E$ incident on the leaves, their other end-point is still in the remaining tree. To approach the problem differently, the depth-first search tree is a vertex cover cover in itself since it contains every vertex. The leaves of the DFS tree are vertices with no unvisited children. So when a leaf is removed any adjecent node in the original graph is still connected to the DFS tree in one way or another.

### 7-23
The diameter of a tree $T = (V, E)$ is given by
$\max_{u,v \in V} \delta(u,v)$
(where $\delta(u,v)$ is the number of edges on the path from $u$ to $v$). Describe an
efficient algorithm to compute the diameter of a tree, and show the correctness
and analyze the running time of your algorithm.
> Below is the algorithm I came up with. The running time is $O(n)$, as every edge is processed exactly once. Regarding the correctness, I think it is pretty straightforward, the exercise is asking this question probably for a different, more clever solution. It is possible to find the diamater using 2 BFSs, the blog post below does a good job to help better understand why this works and also gives a proof by contradiction.
> https://medium.com/@tbadr/tree-diameter-why-does-two-bfs-solution-work-b17ed71d2881

In [13]:
def get_diameter(root):
    if not root:
        return None

    max_diameter = (0, root.val, root.val)

    def find_max_path(node):
        nonlocal max_diameter
    
        if not node.children:
            return (0, node.val)

        max = (0, node.val)
        second_max = (0, node.val)
        for child in node.children:
            child_max_path, start = find_max_path(child)
            longest_path = child_max_path + 1
            if longest_path > max[0]:
                max, second_max = (longest_path, start), max
            elif longest_path > second_max[0]:
                second_max = (longest_path, start)

            total_max = max[0] + second_max[0]
            if total_max > max_diameter[0]:
                max_diameter = (total_max, max[1], second_max[1])
        return max

    find_max_path(root)
    return max_diameter

        
    

In [14]:
# Testing, diamater through root
tree = Node("A")  # root

# Level 1
tree.children = [Node("B"), Node("C")]


# Level 2
tree.children[0].children = [Node("D"), Node("E")]
tree.children[1].children = [Node("F")]


# Level 3
tree.children[0].children[0].children = [Node("G"), Node("H")]
tree.children[0].children[1].children = [Node("I")]


# Level 4 - the farthest leaf nodes (gray-shaded ones in image)
tree.children[0].children[0].children[1].children = [Node("J")]
tree.children[0].children[1].children[0].children = [Node("K"), Node("L")]

# Level 5
tree.children[0].children[0].children[1].children[0].children = [Node("M"), Node("N")]
tree.children[0].children[1].children[0].children[1].children = [Node("O")]

print(f"Testing tree diamater where path is through root : {get_diameter(tree)}") 

# Testing, diamater NOT through root
tree2 = Node("A")  # root

# Level 1
tree2.children = [Node("B"), Node("C")]


# Level 2
tree2.children[0].children = [Node("D"), Node("E")]
tree2.children[1].children = [Node("F")]

# Level 3
tree2.children[0].children[1].children = [Node("G"), Node("H")]
tree2.children[1].children[0].children = [Node("I")]

# Level 4
tree2.children[1].children[0].children[0].children = [Node("J"), Node("K")]

# Level 5
tree2.children[1].children[0].children[0].children[0].children = [Node("L"), Node("M")]

print(f"Testing tree diamater where path is NOT through root : {get_diameter(tree2)}") 

Testing tree diamater where path is through root : (8, 'M', 'O')
Testing tree diamater where path is NOT through root : (8, 'L', 'G')


### 7-34
An *arborescence* of a directed graph $G$ is a rooted tree such that there is a
directed path from the root to every other vertex in the graph. Give an efficient
and correct algorithm to test whether $G$ contains an arborescence, and its time
complexity.